In [0]:
from pyspark.sql.functions import col, to_date, year, month, round,sum, desc,row_number
from pyspark.sql import Window

Read the CSV data from Azure Data Lake

In [0]:
df = spark.read.csv("/mnt/raw/merge_csv.csv", header=True, inferSchema=True)

In [0]:
df.count()

1067371

#Data Cleaning and processing

renaming the column

In [0]:
df = df.withColumnRenamed('Customer ID', 'CustomerID')

In this data, InvoiceNo that start with “C” indicate canceled transactions​, so we are not considering the canceled transactions

In [0]:
df_filterd = df.filter(~col('Invoice').startswith('C'))

Droping rows with null or missing CustomerID

In [0]:
df_filterd = df_filterd.filter(col('CustomerID').isNotNull())


Filter out any records with non-positive Quantity or UnitPrice

In [0]:
df_filterd = df_filterd.filter((col('Quantity') > 0) & (col('Price') > 0))


In [0]:
df_filterd = df_filterd.dropDuplicates()

#  Feature Engineering 

Converted InvoiceDate to a Spark timestamp, then extracted year, month as numeric columns

In [0]:
df_filterd = df_filterd.withColumn('InvoiceDate', to_date(col('InvoiceDate'), 'MM/d/yyyy H:mm'))

df_filterd = df_filterd.withColumn("Year", year(col("InvoiceDate"))) \
    .withColumn("Month", month(col("InvoiceDate")))

Calculated Revenue of each transaction

In [0]:
df_filterd = df_filterd.withColumn('Revenue', round(col('Quantity') * col('Price'),2))

# Aggregations and Window Functions

Monthly Revenue by Product

In [0]:
monthly_sales = df_filterd.groupBy('Year','Month','Stockcode','Description').agg(sum('Revenue').alias('TotalRevenue'))


In [0]:
display(monthly_sales)

Year,Month,Stockcode,Description,TotalRevenue
2009,12,85014B,RED/WHITE DOTS RUFFLED UMBRELLA,5953.95
2010,3,22363,GLASS JAR MARMALADE,155.60000000000002
2010,4,47591C,BLUE FAIRY CAKE CHILD'S APRON,103.35
2010,4,21815,STAR T-LIGHT HOLDER,17.4
2010,2,79323P,PINK CHERRY LIGHTS,1523.1
2010,2,37450,CERAMIC CAKE BOWL + HANGING CAKES,253.20000000000002
2010,4,21327,SKULLS WRITING SET,132.0
2010,5,22088,PAPER BUNTING COLOURED LACE,544.5
2010,6,22548,HEADS AND TAILS SPORTING FUN,111.25
2010,6,22464,HANGING METAL HEART LANTERN,870.1500000000001


Top 10 products per month

In [0]:
window_spec = Window.partitionBy('Year','Month').orderBy(desc('TotalRevenue'))

ranked = monthly_sales.withColumn('Rank', row_number().over(window_spec))



In [0]:
top10_product = ranked.filter(col('Rank') <= 10)

In [0]:
display(top10_product)

Year,Month,Stockcode,Description,TotalRevenue,Rank
2009,12,85123A,WHITE HANGING HEART T-LIGHT HOLDER,16425.509999999995,1
2009,12,15056BL,EDWARDIAN PARASOL BLACK,8632.75,2
2009,12,22086,PAPER CHAIN KIT 50'S CHRISTMAS,7561.200000000003,3
2009,12,84879,ASSORTED COLOUR BIRD ORNAMENT,7271.889999999999,4
2009,12,85099B,JUMBO BAG RED WHITE SPOTTY,6512.700000000001,5
2009,12,22111,SCOTTIE DOG HOT WATER BOTTLE,6498.750000000001,6
2009,12,20679,EDWARDIAN PARASOL RED,6108.25,7
2009,12,79323W,WHITE CHERRY LIGHTS,6069.9,8
2009,12,15056N,EDWARDIAN PARASOL NATURAL,5953.950000000001,9
2009,12,85014B,RED/WHITE DOTS RUFFLED UMBRELLA,5953.95,10


Revenue per country

In [0]:
country_sales = df_filterd.groupBy('Year','Month','Country').agg(sum('Revenue').alias('TotalRevenue')).orderBy(desc('TotalRevenue'))


In [0]:
display(country_sales)

Year,Month,Country,TotalRevenue
2010,11,United Kingdom,977832.810000022
2011,11,United Kingdom,975251.3900000023
2010,10,United Kingdom,875722.8800000112
2011,10,United Kingdom,821220.13
2011,9,United Kingdom,794806.6899999949
2010,9,United Kingdom,679069.9200000012
2009,12,United Kingdom,610346.6299999994
2010,3,United Kingdom,587165.939999997
2011,5,United Kingdom,550359.3499999986
2010,6,United Kingdom,538672.6099999934


 Wrote results back to Proccess layer in Data(Delta) Lake

In [0]:
top10_product.write.format('delta').mode('overwrite').partitionBy('Year','Month').save('/mnt/processed/top10_product')